# Track definition from aerial photography
Generate a parameterized track definition from a masked image based on aerial photography.

The goal of this notebook is to determine the track centreline, track width and length from a pre-masked image, likely based on aerial photography.

# Approach
## Point array of inner and outer contours
Given an image mask of a race track, identify the inner and outer contours of the racing circuit assuming the track is closed.

Let:
* $x_{in}^i, y_{in}^i, i = 1, 2, ..., n$ represent the inner contour of the track
* $x_{out}^j, y_{out}^j, j = 1, 2, ..., m$ represent the outer contour of the track

Where $x_{in}, y_{in}$ and $x_{out}, y_{out}$ are identified using image processing techniques

## Perpendicular direction vector from inner contour
Given a point $x_{in}^i, y_{in}^i$, find the gradient at that point $dx_{in}^i, dy_{in}^i$. The tangent and normal vectors to the point are defined as:

$\vec{t}_{in}^i = [dx_{in}^i, dy_{in}^i]^T$

$\vec{n}_{in}^i = [dy_{in}^i, -dx_{in}^i]^T$

The values of $dx$ and $dy$ are calculated using a central difference.

## Nearest Neighbours
Find the nearest outer neighbour of a given point on the inner contour. Calculate the magnitude of the vector from the outer contour to a given inner contour.

$d = |p_{out} - p_{in}^i|$

$d_{nearest} = \min(d)$


## Identify intercept
Parametrically define the equations of two lines.

$x_{inner} = x_{in}^i + (x_{n,in}^i - x_{in}^i) s$

$y_{inner} = y_{in}^i + (y_{n,in}^i - y_{in}^i) s$

$x_{outer} = x_{out}^i + (x_{out}^{i+1} - x_{out}^i) t$

$y_{inner} = y_{out}^i + (y_{out}^{i+1} - y_{out}^i) t$

If the solution is $0 < t < 1$, this means the intersection lies between the outer point and its adjacent point.

If the solution is not $0 < t < 1$, try the other adjacent point on the outer contour. If this fails, then reject the nearest neighbour and find the next closest point.

In [ ]:
%matplotlib notebook

import cv2
import scipy.signal
import numpy as np
import matplotlib.pyplot as plt
import pykalman

# Parallelism
import time
import psutil 
from joblib import Parallel, delayed

cpus = psutil.cpu_count(logical=False)

## Read the initial image and find inner/outer track limits
Open a pre-masked image and process it with OpenCV. The track surface is represented by the pixels in black, and non-track surfaces are represented by the pixles in white.

The goal using OpenCV is to:
* Read in the image
* Identify the inner and outer contours of the race track
* Filter and decimate the data points representing the contours

In [ ]:
img = cv2.imread('TMP.png', 0)
img = img[::-1]  # flip for +E, +N to correspond to +x, +y

In [ ]:
plt.figure()
plt.imshow(img)
plt.gca().invert_yaxis()
plt.show()

In [ ]:
# Initialize result
track_def = dict()

In [ ]:
# Find contours
contours, hierarchy = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
track_def['points_inner'] = np.array([(z[0][0], z[0][1]) for z in contours[2]])
track_def['points_outer'] = np.array([(z[0][0], z[0][1]) for z in contours[1]])

In [ ]:
def decimate_and_filter(signal, factor, ftype, n):
    return scipy.signal.decimate(signal.astype(np.float32), factor, ftype=ftype, n=n)


def decimate_and_filter_contour(contour, factor, ftype, n):
    x, y = contour.T
    return np.column_stack((decimate_and_filter(x, factor, ftype, n),
                            decimate_and_filter(y, factor, ftype, n)))

# Decimation filter design
filter_type = 'iir'
filter_order = 5

# Filter and decimate contour signals
track_def['points_inner'] = decimate_and_filter_contour(track_def['points_inner'],
                                                        10,
                                                        filter_type,
                                                        filter_order)

track_def['points_outer'] = decimate_and_filter_contour(track_def['points_outer'],
                                                        5,
                                                        filter_type,
                                                        filter_order)

In [ ]:
plt.figure()

plt.scatter(track_def['points_inner'].T[0], track_def['points_inner'].T[1], s=2)
plt.scatter(track_def['points_outer'].T[0], track_def['points_outer'].T[1], s=2)

plt.axis('scaled')
plt.show()

## Find the gradient of the inner contour


In [ ]:
def gradient_closed_array(array):
    # Calculates the gradient of an array using central differences
    # Corrects the gradient at the tails assuming array is circular
    gradient = np.gradient(array)
    gradient[0] = 0.5 * (array[1] - array[-1])
    gradient[-1] = 0.5 * (array[0] - array[-2])
    return gradient


def gradient_closed_contour(contour):
    x, y = contour.T
    return np.column_stack((gradient_closed_array(x),
                            gradient_closed_array(y)))
    

track_def['dpoints_inner'] = gradient_closed_contour(track_def['points_inner'])
track_def['dpoints_outer'] = gradient_closed_contour(track_def['points_outer'])

In [ ]:
plt.figure()
plt.plot(track_def['points_outer'].T[0])
plt.show()

## Find the opposing point on the outer contour
Given an inner point, find the nearest outer neighbour and check if the perpendicular line intersects between any adjacent points. If the intersection does not lie between the two outer points, recursively call the algorithm rejecting the current nearest neighbour.

In [ ]:
def find_parametric_intersection(p1, p2, p3, p4):
    p43 = p4 - p3
    p31 = p3 - p1
    p21 = p2 - p1
    
    s = (p43[0] * p31[1] - p31[0] * p43[1]) / (p43[0] * p21[1] - p21[0] * p43[1])
    t = (p21[0] * p31[1] - p31[0] * p21[1]) / (p43[0] * p21[1] - p21[0] * p43[1])

    return (s, t)
  
    
def find_opposing_point(inner_point, inner_derivative, outer_points, indices):
    """ Finds the perpendicular vector that intersects the outer points, recusively"""
    # Vector perpendicular to the derivative
    inner_perpendicular = np.copy(inner_derivative[::-1])
    inner_perpendicular[1] *= -1
    
    # Compute the distance from the inner point to all the outer points
    direction_magnitudes = np.array([np.linalg.norm(vector) for vector in outer_points[indices] - inner_point])
    
    # Find the nearest neighbour  
    array_index = np.argmin(direction_magnitudes)
    position_index = indices[array_index]
    outer_point = outer_points[position_index]
    
    for adjacent in [-1, 1]:
        # Find the adjacent point to the selected nearest neighbour
        adjacent_point = outer_points[(position_index+adjacent) % len(outer_points)]

        # Find the intersection
        s, t = find_parametric_intersection(inner_point,
                                            inner_point + inner_perpendicular,
                                            outer_point,
                                            adjacent_point)
        
        # Return if perpendicular line inserects with the outer section
        if (0 < t < 1):
            return inner_point + inner_perpendicular * s
    
    # Recursively find the opposing point
    return find_opposing_point(inner_point, 
                               inner_derivative, 
                               outer_points,
                               np.delete(indices, array_index, axis=0))
    
# Preprocessing
inner_points = track_def['points_inner']
outer_points = track_def['points_outer']
inner_derivatives = track_def['dpoints_inner']

# Calculate the 'opposing' points
# Parallelism
indices = np.arange(0, len(outer_points))
arg_instances = [(inner_points[idx], inner_derivatives[idx], outer_points, indices) for idx, _ in enumerate(inner_points)]
opposing_points = Parallel(n_jobs=cpus)((delayed(find_opposing_point)(*args) for args in arg_instances))

# opposing_points = np.empty(inner_points.shape)
# for idx, _ in enumerate(opposing_points):
#     indices = np.arange(0, len(outer_points))
#     opposing_points[idx] = find_opposing_point(inner_points[idx], inner_derivatives[idx], outer_points, indices)

# Post processing of parameters
track_def['points_opposing'] = np.array(opposing_points)
track_def['points_centreline'] = 0.5 * (track_def['points_opposing'] + track_def['points_inner'])
track_def['track_width'] = np.array([np.linalg.norm(vector) for vector in track_def['points_opposing'] - track_def['points_inner']])
track_def['track_dist'] = np.concatenate(([0], np.cumsum(np.sqrt(np.sum(np.diff(track_def['points_centreline'], axis=0)**2, axis=1)))))

In [ ]:
# Generate an initial estimate of the track curvature
ds = gradient_closed_array(track_def['track_dist'])
dcx = gradient_closed_array(track_def['points_centreline'].T[0])
ddcx = gradient_closed_array(dcx)
dcy = gradient_closed_array(track_def['points_centreline'].T[1])
ddcy = gradient_closed_array(dcy)

# Calculate derivatives
dcx_ds = dcx / ds
dcy_ds = dcy / ds
d2cx_dds2 = ddcx / ds / ds
d2cy_dds2 = ddcy / ds / ds

track_def['curvature'] = (dcx_ds * d2cy_dds2 - dcy_ds * d2cx_dds2) / (dcx_ds**2 + dcy_ds**2) ** 1.5

In [ ]:
plt.figure()
plt.plot(track_def['track_dist'], track_def['curvature'])
plt.show()

plt.figure()
plt.plot(track_def['track_dist'], track_def['track_width'])
plt.show()

## Visualize results with source image

In [ ]:
ref_img = cv2.imread('USGS_IMAGERY_TORONTO_MOTORSPORTS_PARK_2008_CROP.png')
ref_img = ref_img[::-1]  # flip for +E, +N to correspond to +x, +y

plt.figure()
plt.imshow(ref_img, alpha=0.7)
plt.gca().invert_yaxis()

xs = np.column_stack((track_def['points_inner'].T[0], track_def['points_opposing'].T[0]))
ys = np.column_stack((track_def['points_inner'].T[1], track_def['points_opposing'].T[1]))

factor = 5
plt.plot(xs, ys, c='black')
plt.plot(xs[::factor].T, ys[::factor].T, c='gray')
plt.plot(track_def['points_centreline'].T[0], track_def['points_centreline'].T[1])
plt.plot(xs[0].T, ys[0].T, c='red')

plt.show()